### PyMuPDF4LLM

Pymupdf4llm 공식문서 링크입니다.

https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/api.html#pymupdf4llm-api

In [1]:
# %pip install pymupdf4llm

In [ ]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("data/arxiv_paper.pdf")

In [ ]:
# Pymupdf4llm이 파싱한 결과를 md 파일로 저장합니다.
import pathlib
pathlib.Path("pymupdf4llm_AI.md").write_bytes(md_text.encode())

In [ ]:
import pymupdf4llm
#파일의 일부분만 추출하여 마크다운 변환할 수 있습니다.
md_text = pymupdf4llm.to_markdown(
    "data/arxiv_paper.pdf",
    pages=[0, 1, 2], 
    hdr_info=False,
    ignore_code=True
)

print(md_text)

#### VLM에 캡셔닝 작업을 전달하기 전에 정보량이 풍부한 이미지만 남기기

이제 EasyOCR을 활용해 이미지 속 텍스트의 수를 기준으로 정보량이 풍부한 이미지와 그렇지 않은 이미지를 구분할 수 있습니다.

그 이후는 VLM 추론을 통해 이미지 캡션을 수행하면 됩니다.

In [ ]:
# %pip install easyocr

In [ ]:
md_data = pymupdf4llm.to_markdown(
    "data/국가별 공공부문 AI 도입 및 활용 전략.pdf",
    page_chunks=True,
    write_images=True,
    image_path="PyMuPDF4LLM_AI",       # 저장 경로 지정
    image_format="png",          # 저장 포맷
    dpi=200,
    image_size_limit=0.02        # 2% 이하 크기 이미지 제외
)

EasyOCR을 통해 이미지 내 글자를 인식하고, 글자수가 충분한 경우에만 VLM에 넘기도록 처리하는 로직입니다.

In [ ]:
import os
from PIL import Image
import numpy as np
import easyocr

# EasyOCR reader
reader = easyocr.Reader(['ko', 'en'], gpu=True)

# 이미지 폴더 경로
image_folder = "output_images"

# 결과 저장용 리스트
rich_images = []   # 정보 풍부
poor_images = []   # 정보 부족

# 폴더 내 이미지 순회
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(image_folder, filename)

        try:
            # 이미지 불러오기
            img = Image.open(img_path).convert("RGB")
            img_np = np.array(img)

            # OCR 수행
            results = reader.readtext(img_np)

            texts = [res[1] for res in results]
            total_text = ''.join(texts)

            # 조건 검사
            if len(texts) >= 3 and len(total_text) >= 30:
                rich_images.append((filename, len(texts), len(total_text)))
            else:
                poor_images.append((filename, len(texts), len(total_text)))

        except Exception as e:
            print(f"❌ 오류 발생 - {filename}: {e}")

# 결과 출력
print("\n📘 정보량 많은 이미지:")
for name, blocks, chars in rich_images:
    print(f"✔ {name} - 블럭 {blocks}개, 총 글자수 {chars}")

print("\n📄 정보 부족한 이미지:")
for name, blocks, chars in poor_images:
    print(f"✖ {name} - 블럭 {blocks}개, 총 글자수 {chars}")
